In [1]:
import tensorflow as tf
import numpy as np
import json
from collections import defaultdict
from tokenizer import tokenize

In [2]:
# reviews_path = '/Users/nateparrott/Downloads/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'



In [3]:
# counts = defaultdict(int)
# review_lengths = []

# reviews_path = '/Users/nateparrott/Downloads/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'
# for line in open(reviews_path):
#     item = json.loads(line)
#     if item['type'] == 'review':
#         tokens = tokenize(item['text'])
#         rating = item['stars']
#         review_lengths.append(len(tokens))
#         for tk in tokens: counts[tk] += 1
#     if len(review_lengths) > 100 * 1000:
#         break

In [4]:
# review_lengths.sort()
# print '90th percentile review length:', review_lengths[int(len(review_lengths) * 0.9)]
# print '70th percentile review length:', review_lengths[int(len(review_lengths) * 0.7)]
# vocab = [w for w, c in counts.iteritems() if c > 50]
# print 'Vocab size:', len(vocab)

In [5]:
char_lengths = []

for line in open(reviews_path):
    item = json.loads(line)
    if item['type'] == 'review':
        char_lengths.append(len(item['text']))
    if len(char_lengths) > 100 * 1000:
        break

IOError: [Errno 2] No such file or directory: '/Users/nateparrott/Downloads/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'

In [20]:
char_lengths.sort()
print '90th percentile review length (chars):', char_lengths[int(len(char_lengths) * 0.9)]
print '70th percentile review length (chars):', char_lengths[int(len(char_lengths) * 0.7)]


90th percentile review length (chars): 1303
70th percentile review length (chars): 713


In [21]:
max_review_len = 1000

In [30]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def pad(chars, length):
    chars = chars[:min(len(chars), length)]
    return chars + ' ' * (length - len(chars))

def convert(filename, skip=0, count=None, out_path=None):
    writer = tf.python_io.TFRecordWriter(out_path)
    i = 0
    for line in open(filename):
        item = json.loads(line)
        if item['type'] == 'review':
            if i >= skip:
                chars = item['text'].encode('ascii', 'ignore')
                chars = pad(chars, max_review_len)
                fdict = {'stars': _int64_feature(item['stars']), 'text': _bytes_feature(chars)}
                example = tf.train.Example(features=tf.train.Features(feature=fdict))
                writer.write(example.SerializeToString())
            i += 1
            if count is not None and i - skip > count: break
    writer.close()

test_size = 10 * 1000
train_size = 10 * 1000
convert(reviews_path, 0, test_size, 'yelp_test.tfrecords')
convert(reviews_path, test_size, train_size, 'yelp_train.tfrecords')